In [3]:
from pprint import pprint
import numpy as np
import random
from collections import defaultdict
from tqdm import tqdm
from joblib import Parallel, delayed
import networkx as nx

In [18]:
def get_icd10_codes():
    icd10 = defaultdict()
    with open('../results/icd10cm_codes_2018.txt', 'r+') as f:
        for line in f.readlines():
            line = line.split()
            icd10[line[0].lower()] = ' '.join(line[1:]).lower()
        return icd10

def retrieve_titles_for_subgraph(graph, icd10, min_nodes=2):
    subgraphs = list(nx.connected_component_subgraphs(graph))
    results = defaultdict(list)
    for subgraph in subgraphs:
        nodes = subgraph.nodes()
        if len(nodes) >= min_nodes:
            results[', '.join(nodes.keys())] = [icd10[node] for node in nodes]
    return results

In [28]:
def get_overlap(a, b):
    results = defaultdict(list)
    for key1 in a.keys():
        codes = set(key1.split(', '))
        codes_intersection = [codes.intersection(key2.split(', ')) for key2 in b.keys() if len(codes.intersection(key2.split(', '))) > 1]
        if codes_intersection:
            results[key1] = codes_intersection
    return results

GloVe

In [20]:
glove50 = nx.read_edgelist('glove_analysis/50/graph_glove_200')
glove150 = nx.read_edgelist('glove_analysis/150/graph_glove_200')

In [29]:
described_glove50 = retrieve_titles_for_subgraph(glove50, get_icd10_codes())
described_glove150 = retrieve_titles_for_subgraph(glove150, get_icd10_codes())

In [30]:
len(described_glove50), len(described_glove150)

(259, 243)

In [32]:
3described_glove50

defaultdict(list,
            {'a09, r310, c321': ['infectious gastroenteritis and colitis, unspecified',
              'gross hematuria',
              'malignant neoplasm of supraglottis'],
             'a156, n812': ['tuberculous pleurisy',
              'incomplete uterovaginal prolapse'],
             'a157, p810': ['primary respiratory tuberculosis',
              'environmental hyperthermia of newborn'],
             'a190, l550': ['acute miliary tuberculosis of a single specified site',
              'sunburn of first degree'],
             'a200, g311': ['bubonic plague',
              'senile degeneration of brain, not elsewhere classified'],
             'a222, d269, h36': ['gastrointestinal anthrax',
              'other benign neoplasm of uterus, unspecified',
              'retinal disorders in diseases classified elsewhere'],
             'a310, d098': ['pulmonary mycobacterial infection',
              'carcinoma in situ of other specified sites'],
             'a33, l1

In [31]:
len(get_overlap(described_glove50, described_glove150))

2

Doc2vec

In [6]:
doc2vec50 = nx.read_edgelist('doc2vec_analysis/cbow/50/graph_200')
doc2vec150 = nx.read_edgelist('doc2vec_analysis/cbow/150/graph_200')

In [7]:
described_doc2vec50 = retrieve_titles_for_subgraph(doc2vec50, get_icd10_codes())
described_doc2vec150 = retrieve_titles_for_subgraph(doc2vec150, get_icd10_codes())

In [8]:
len(described_doc2vec50), len(described_doc2vec150)

(219, 219)

In [9]:
len(set(described_doc2vec50).out(set(described_doc2vec150)))

217

In [ ]:
# icd10 = get_icd10_codes()
# with open('overlap', 'w+') as output_file:  
#     for _, codes in overlap.items():
#         output_file.write('{}:\n'.format(', '.join(codes[0])))
#         output_file.write('{}\n'.format('\n'.join([icd10[code] for code in codes[0]])))
#         output_file.write('\n')